In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
import os
import pickle
import uuid

def document_chunker_langchain(directory_path,
                                model_name="sentence-transformers/all-MiniLM-L6-v2",
                                chunk_size=1024,
                                chunk_overlap=0):
    """
    Processes files in a directory, chunks text based on the model's tokenization constraints, and
    prepares a vector store with the text chunks and metadata using LangChain.
    """
    # Initialize embeddings model and text splitter
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    documents = []  # List to hold all documents

    # Iterate through files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            base = os.path.basename(file_path)
            sku = os.path.splitext(base)[0]

            
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()

            # Split text into manageable chunks
            chunks = text_splitter.split_text(text)

            # Create Document objects with metadata
            for chunk in chunks:
                doc = Document(
                    page_content=chunk,
                    metadata={"file_name": sku}
                )
                documents.append(doc)

    # Build a vector store from the documents
    i = 1
    # print(documents[i].page_content)
    vector_store = FAISS.from_documents(documents, embeddings)
    num_vectors = vector_store.index.ntotal

    # recon = vector_store.index.reconstruct(i)
    # recon = vector_store.index
    print('Vector: ',num_vectors)
    doc_id = list(vector_store.docstore._dict.keys())[i]
    document = vector_store.docstore._dict[doc_id]
    # print(f"Metadata: {document.metadata}")
    # print('list(vector_store.docstore._dict.keys())[i]',list(vector_store.docstore._dict.keys())[i])
    pickle.dump(vector_store, open("./pickleDump/vector_store.pkl", "wb"))
    return vector_store

# Example usage
# directory_path = "./Document/test"
vector_store = document_chunker_langchain("./Document/test")


# print(vector_store)
# Querying the vector store
query = "bulb"
results = vector_store.similarity_search(query, k=5)
# viecpickle.load(open("./pickleDump/vector_store.pkl", "rb"))

# for result in results:
#     print(f"Text: {result.page_content}\nMetadata: {result.metadata}\n")




Vector:  7


FileNotFoundError: [Errno 2] No such file or directory: './pickleDump/vector_store.pkl'

In [2]:
def construct_prompt(system_prompt, retrieved_docs, user_query):
    prompt = f"""{system_prompt}

    Here is the retrieved context:
    {retrieved_docs}

    Here is the users query:
    {user_query}
    """
    return prompt

system_prompt = """
You are an intelligent search engine. You will be provided with some retrieved context, as well as the users query.

Your job is to understand the request, and answer based on the retrieved context.
"""

retrieved_docs = """
Wall-mounted electric fireplace with realistic LED flames and heat settings. Features a black glass frame and remote control for easy operation. Ideal for adding warmth and ambiance. Manufactured by Hearth & Home. Dimensions: 50"W x 6"D x 21"H.
"""

prompt = construct_prompt(system_prompt=system_prompt,
                          retrieved_docs=retrieved_docs,
                          user_query="bulb")

In [9]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

def construct_prompt(system_prompt, retrieved_docs, user_query):
    # Define the system message prompt
    system_message = SystemMessagePromptTemplate.from_template(system_prompt)
    
    # Define the human message prompt with placeholders
    human_message = HumanMessagePromptTemplate.from_template("""
                                Here is the retrieved context:
                                {retrieved_docs}

                                Here is the user's query:
                                {user_query}
                                    """)
    
    # Combine the system and human messages into a ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    
    # Format the chat prompt with the provided inputs
    formatted_prompt = chat_prompt.format_messages(
        retrieved_docs=retrieved_docs,
        user_query=user_query
    )
    
    return formatted_prompt

# Inputs
system_prompt = """
You are an intelligent search engine. You will be provided with some retrieved context, as well as the users query.

Your job is to understand the request, and answer based on the retrieved context.
"""

retrieved_docs = """
Wall-mounted electric fireplace with realistic LED flames and heat settings. Features a black glass frame and remote control for easy operation. Ideal for adding warmth and ambiance. Manufactured by Hearth & Home. Dimensions: 50"W x 6"D x 21"H.
"""

user_query = "What are the dimensions of the fireplace?"

# Generate the prompt
formatted_prompt = construct_prompt(system_prompt, retrieved_docs, user_query)
# Display the formatted prompt
for message in formatted_prompt:
    print(f"{message.type.upper()} MESSAGE:")
    print(message.content)
    print()

SYSTEM MESSAGE:

You are an intelligent search engine. You will be provided with some retrieved context, as well as the users query.

Your job is to understand the request, and answer based on the retrieved context.


HUMAN MESSAGE:

                                Here is the retrieved context:
                                
Wall-mounted electric fireplace with realistic LED flames and heat settings. Features a black glass frame and remote control for easy operation. Ideal for adding warmth and ambiance. Manufactured by Hearth & Home. Dimensions: 50"W x 6"D x 21"H.


                                Here is the user's query:
                                What are the dimensions of the fireplace?
                                    



In [11]:
chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

SyntaxError: invalid decimal literal (2149946735.py, line 1)

SyntaxError: invalid syntax (1414495550.py, line 1)

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOllama(
    model="tinyllama",
    temperature=0,
)

chain = {"docs": format_docs} | formatted_prompt | llm | StrOutputParser()


# messages = [
#     (
#         "system",
#         "You are a helpful assistant that translates English to French. Translate the user sentence.",
#     ),
#     ("human", "I love programming."),
# ]

results = vector_store.similarity_search("bulb", k=5)

chain.invoke(results)

# llm.invoke(messages)

NameError: name 'formatted_prompt' is not defined

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | formatted_prompt | llm | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

results = vector_store.similarity_search(question, k=5)

chain.invoke(results)

TypeError: unsupported operand type(s) for |: 'dict' and 'list'

In [ ]:
wget https://huggingface.co/jartine/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile


SyntaxError: invalid decimal literal (3897215599.py, line 1)

## vector content

In [71]:
num_vectors = vector_store.index.ntotal

# Iterate through all stored vectors and their metadata
for i in range(num_vectors):
    # Retrieve the vector
    vector = vector_store.index.reconstruct(i)  # Get the embedding vector
    # Retrieve the corresponding document metadata
    doc_id_key = list(vector_store.docstore._dict.keys())[i]
    doc_id_value = list(vector_store.docstore._dict.values())[i]
    document = vector_store.docstore._dict[doc_id_key]
    
    # Print details
    print(f"Document ID: {doc_id_key}")
    print(f"document_content: {doc_id_value}")
    print(f"Metadata: {document.metadata}")
    print(f"Vector: {vector}")
    print("\n---\n")

Document ID: 0a44b13f-57c5-4b0d-aff5-8f9328554fc8
document_content: page_content='{
  "MBR-2001": "Traditional sleigh bed crafted in rich walnut wood, featuring a curved headboard and footboard with intricate grain details. Queen size, includes a plush, supportive mattress. Produced by Heritage Bed Co. Dimensions: 65\"W x 85\"L x 50\"H.",
  "MBR-2002": "Art Deco-inspired vanity table in a polished ebony finish, featuring a tri-fold mirror and five drawers with crystal knobs. Includes a matching stool upholstered in silver velvet. Made by Luxe Interiors. Vanity dimensions: 48\"W x 20\"D x 30\"H, Stool dimensions: 22\"W x 16\"D x 18\"H.",
  "MBR-2003": "Set of sheer linen drapes in soft ivory, offering a delicate and airy touch to bedroom windows. Each panel measures 54\"W x 84\"L. Features hidden tabs for easy hanging. Manufactured by Tranquil Home Textiles.",' metadata={'file_name': 'test_data'}
Metadata: {'file_name': 'test_data'}
Vector: [ 2.96612475e-02 -4.58519533e-02  2.86810286e-

In [20]:
pip install sentence-transformers

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------------- ---------------------- 4.7/11.0 MB 28.6 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.0 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 18.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   -- ------------------------------------- 2.4/44.8 MB 12.2 MB/s eta 0:00:04
   ---- ----------------------------------- 5.0/44.8 MB 12.1 MB/s eta 0:00:04
   ------ --------------------------------- 7.3/44.8 MB 11.9 MB/s eta 0:00:04
   ------- -------------------------------- 8.1/44.8 MB 10.3 MB/s eta 0:00:04
   --------- ------------------------------ 11.0/44.8 MB 10.4 MB/s eta 0:00:04
   ----------- ---------------------------- 13.4/44.8 MB 10.5 MB/s eta 0:00:03
   -------------- ------------------------- 15.7/44.8 MB 10.6 MB/s eta 0:00:03
   --------------- ------------------------ 17.3/44.8 MB 10.5 MB/s eta 0:00:

In [22]:
docs = document_chunker_langchain(directory_path='./Document/test',
                        model_name='sentence-transformers/all-MiniLM-L6-v2',
                        chunk_size=256)
docs

In [23]:
keys = list(docs.keys())
print(len(docs))
print(docs[keys[0]])

AttributeError: 'FAISS' object has no attribute 'keys'

In [9]:
from transformers import AutoModel, AutoTokenizer

model_name = "BAAI/bge-small-en-v1.5"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer.save_pretrained("model/tokenizer")
model.save_pretrained("model/embedding")

In [10]:
def compute_embeddings(text):
    tokenizer = AutoTokenizer.from_pretrained("/model/tokenizer") 
    model = AutoModel.from_pretrained("/model/embedding")

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True) 
    
    # Generate the embeddings 
    with torch.no_grad():    
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze()

    return embeddings.tolist()

In [11]:
def create_vector_store(doc_store):
    vector_store = {}
    for doc_id, chunks in doc_store.items():
        doc_vectors = {}
        for chunk_id, chunk_dict in chunks.items():
            # Generate an embedding for each chunk of text
            doc_vectors[chunk_id] = compute_embeddings(chunk_dict.get("text"))
        # Store the document's chunk embeddings mapped by their chunk UUIDs
        vector_store[doc_id] = doc_vectors
    return vector_store

In [28]:
def compute_matches(vector_store, query_str, top_k):
    """
    This function takes in a vector store dictionary, a query string, and an int 'top_k'.
    It computes embeddings for the query string and then calculates the cosine similarity against every chunk embedding in the dictionary.
    The top_k matches are returned based on the highest similarity scores.
    """
    # Get the embedding for the query string
    query_str_embedding = np.array(compute_embeddings(query_str))
    scores = {}

    # Calculate the cosine similarity between the query embedding and each chunk's embedding
    for doc_id, chunks in vector_store.items():
        for chunk_id, chunk_embedding in chunks.items():
            chunk_embedding_array = np.array(chunk_embedding)
            # Normalize embeddings to unit vectors for cosine similarity calculation
            norm_query = np.linalg.norm(query_str_embedding)
            norm_chunk = np.linalg.norm(chunk_embedding_array)
            if norm_query == 0 or norm_chunk == 0:
                # Avoid division by zero
                score = 0
            else:
                score = np.dot(chunk_embedding_array, query_str_embedding) / (norm_query * norm_chunk)

            # Store the score along with a reference to both the document and the chunk
            scores[(doc_id, chunk_id)] = score

    # Sort scores and return the top_k results
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)[:top_k]
    top_results = [(doc_id, chunk_id, score) for ((doc_id, chunk_id), score) in sorted_scores]

    return top_results

In [72]:
matches = compute_matches(vector_store=vector_store,
                            query_str="Wall-mounted electric fireplace with realistic LED flames",
                            top_k=3)

NameError: name 'np' is not defined

In [3]:
from llama_cpp import Llama
import sys

def stream_and_buffer(base_prompt, llm, max_tokens=800, stop=["Q:", "\n"], echo=True, stream=True):

    # Formatting the base prompt
    formatted_prompt = f"Q: {base_prompt} A: "

    # Streaming the response from llm
    response = llm(formatted_prompt, max_tokens=max_tokens, stop=stop, echo=echo, stream=stream)

    buffer = ""

    for message in response:
        chunk = message['choices'][0]['text']
        buffer += chunk

        words = buffer.split(' ')
        for word in words[:-1]:  
            sys.stdout.write(word + ' ')  
            sys.stdout.flush()  

       
        buffer = words[-1]

    if buffer:
        sys.stdout.write(buffer)
        sys.stdout.flush()
    print(buffer)
def construct_prompt(system_prompt, retrieved_docs, user_query):
    prompt = f"""{system_prompt}

    Here is the retrieved context:
    {retrieved_docs}

    Here is the users query:
    {user_query}
    """
    return prompt

system_prompt = """
You are an intelligent search engine. You will be provided with some retrieved context, as well as the users query.

Your job is to understand the request, and answer based on the retrieved context.
"""

retrieved_docs = """
Wall-mounted electric fireplace with realistic LED flames and heat settings. Features a black glass frame and remote control for easy operation. Ideal for adding warmth and ambiance. Manufactured by Hearth & Home. Dimensions: 50"W x 6"D x 21"H.
"""

prompt = construct_prompt(system_prompt=system_prompt,
                          retrieved_docs=retrieved_docs,
                          user_query="bulb")

llm = Llama(model_path="./llama_model/tinyllama-1.1b-chat-v1.0.Q8_0.gguf", n_gpu_layers=1)

stream_and_buffer(prompt, llm)


llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from ./llama_model/tinyllama-1.1b-chat-v1.0.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632


llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   8:              llama.attention.head_count_kv u32              = 4
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  10:                       llama.rope.freq_base f32              = 10000.000000
llama_model_loader: - kv  11:                          general.file_type u32              = 7
llama_model_loader: - kv  12:                       tokenizer.ggml.model str              = llama
llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  token

In [7]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

In [3]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a gentleman",
    },
    {"role": "user", "content": "Where is mysore located?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


<|system|>
You are a friendly chatbot who always responds in the style of a gentleman</s>
<|user|>
Where is mysore located?</s>
<|assistant|>
Myśro is located in the Indian state of Karnataka, primarily in the Mandya district. The district is known for its rich cultural heritage and history, and is a popular destination for tourists and travelers. The city is located about 120 kilometers southeast of Bangalore, the state capital.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

def construct_prompt(system_prompt, retrieved_docs, user_query):
    # Define the system message prompt
    system_message = SystemMessagePromptTemplate.from_template(system_prompt)
    
    # Define the human message prompt with placeholders
    human_message = HumanMessagePromptTemplate.from_template("""
                                Here is the retrieved context:
                                {retrieved_docs}

                                Here is the user's query:
                                {user_query}
                                    """)
    
    # Combine the system and human messages into a ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    
    # Format the chat prompt with the provided inputs
    formatted_prompt = chat_prompt.format_messages(
        retrieved_docs=retrieved_docs,
        user_query=user_query
    )
    
    return formatted_prompt

# Inputs
system_prompt = """
You are an intelligent search engine. You will be provided with some retrieved context, as well as the users query.

Your job is to understand the request, and answer based on the retrieved context.
"""

retrieved_docs = """
Wall-mounted electric fireplace with realistic LED flames and heat settings. Features a black glass frame and remote control for easy operation. Ideal for adding warmth and ambiance. Manufactured by Hearth & Home. Dimensions: 50"W x 6"D x 21"H.
"""

user_query = "What are the dimensions of the fireplace?"

# Generate the prompt
formatted_prompt = construct_prompt(system_prompt, retrieved_docs, user_query)
# Display the formatted prompt
for message in formatted_prompt:
    print(f"{message.type.upper()} MESSAGE:")
    print(message.content)
    print()

SYSTEM MESSAGE:

You are an intelligent search engine. You will be provided with some retrieved context, as well as the users query.

Your job is to understand the request, and answer based on the retrieved context.


HUMAN MESSAGE:

Here is the retrieved context:

Wall-mounted electric fireplace with realistic LED flames and heat settings. Features a black glass frame and remote control for easy operation. Ideal for adding warmth and ambiance. Manufactured by Hearth & Home. Dimensions: 50"W x 6"D x 21"H.


Here is the user's query:
What are the dimensions of the fireplace?
    



In [4]:
import os
import pickle
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_ollama import ChatOllama
from pypdf import PdfReader  


def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using pypdf."""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()  
    return text

def build_vector_store(directory_path, model_name="sentence-transformers/all-MiniLM-L6-v2", chunk_size=1024, chunk_overlap=0):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=int(os.environ.get("CHUNK_SIZE", 1024)), chunk_overlap=chunk_overlap)
    documents = []

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            base = os.path.basename(file_path)
            sku = os.path.splitext(base)[0]
            
            # Check file type and extract text accordingly
            if filename.endswith(".pdf"):
                text = extract_text_from_pdf(file_path)
            elif filename.endswith(".txt"):
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()
            else:
                print(f"Unsupported file type for {filename}, skipping.")
                continue
            
            chunks = text_splitter.split_text(text)
            for chunk in chunks:
                doc = Document(page_content=chunk, metadata={"file_name": sku})
                documents.append(doc)

    vector_store = FAISS.from_documents(documents, embeddings)
    # pickle.dump(vector_store, open("./pickleDump/vector_store.pkl", "wb"))
    return vector_store

def construct_prompt(system_prompt, retrieved_docs, user_query):
    system_message = SystemMessagePromptTemplate.from_template(system_prompt)
    human_message = HumanMessagePromptTemplate.from_template(
        """
        Here is the retrieved context:
        {retrieved_docs}

        Here is the user's query:
        {user_query}
        """
    )
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    formatted_prompt = chat_prompt.format_messages(retrieved_docs=retrieved_docs, user_query=user_query)
    return formatted_prompt

def query_llm(vector_store, query, llm_model="llama3.2", temperature=0):
    results = vector_store.similarity_search(query, k=5)
    print('retrieved_docs', results)
    retrieved_docs = "\n\n".join([doc.page_content for doc in results])
    system_prompt = """
    You are a highly intelligent assistant designed to answer user questions strictly based on the provided document context. Here are your responsibilities:

    1. **Use Provided Context Only**: Your answers must rely exclusively on the content retrieved from the document. Do not infer or generate information outside the document's scope.

    2. **Handle Irrelevant Queries**: If the user's query is unrelated to the provided document, respond with: 
    "The query is not relevant to the document."

    3. **Include Page Numbers**: 
    - If the answer is based on the document, include the relevant page number(s) to indicate where the information was sourced.
    - If the query is irrelevant, mention that the document context provided does not address the query and include the retrieved page numbers.

    4. **Structured Output**:
    - If the answer is based on the document, include specific details and context to ensure accuracy, along with the page number(s).
    - If the query is irrelevant, explicitly state it without additional assumptions, mentioning the retrieved page numbers.

    5. **Tone and Clarity**: Always maintain a professional and clear tone.

    6. **Content Relevance**: If you feel the retrieved document and query are relevant, answer with an appropriate and clear response, referencing the specific page(s).

    You will be provided with:
    - **Retrieved Document Context**: The content relevant to the user's query, along with page number(s).
    - **User Query**: The question or task from the user.

    Your task:
    - Analyze the retrieved document context and its page numbers.
    - Answer the query based on the document, referencing the page number(s), or identify the query as irrelevant if the document does not contain an answer.
    
    """

    formatted_prompt = construct_prompt(system_prompt, retrieved_docs, query)
    llm = ChatOllama(model=llm_model, temperature=temperature)
    final_prompt = "".join([message.content for message in formatted_prompt])
    response = llm.invoke(final_prompt)
    return response


In [3]:
if __name__ == "__main__":
    if not os.path.exists("./pickleDump/vector_store.pkl"):
        vector_store = build_vector_store("./Document/test")
        print('not there',vector_store)
    else:
        vector_store = pickle.load(open("./pickleDump/vector_store.pkl", "rb"))
        print('there',vector_store)
    
    user_query = "Which device is capable of streaming music"

    response = query_llm(vector_store, user_query)
    print("LLM Response:")
    print(response)


C:\Users\nanupam\AppData\Local\Temp\ipykernel_20952\1277766130.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
d:\Anupam Project\GenAI_v2\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [5]:
if __name__ == "__main__":
    print('starting with vector store')
    vector_store = build_vector_store("./Document/test")
    print('done with vector store')
    user_query = "How to download and install MFP agent...?" 
    print('started with llm')
    response = query_llm(vector_store, user_query)
    print("LLM Response:")
    print(response)

starting with vector store
done with vector store
started with llm
retrieved_docs [Document(metadata={'file_name': 'SHARP Synappx GO Documentation (2)'}, page_content='Now see the device is assigned or not if not go to the bottom and check \nthe MFP IP Address is in unassigned if its in that click on unassigned \nand remove or delete unassigned devices. Now go check whether its \nadded to your agent or not, if added go to workspaces and open your \nlicensed workspace and check if your pc name or agent name is added or \nnot. \n \nThen, \n \nSee the above picture for your clarification. \nNow Select MFPs to associate with this workspace and confirm it on \nclicking ok button on bottom. See the next image that will tell you the \nMFP is assigned to your workspace. \n \nMFP is successfully Connected to your Workspace. \n \nNext, \n \n \nINSTALL THE MOBILE APPLICATION IN YOUR MOBILE \nPHONE \nInstall the same stack mobile application in your mobile phone are in \nyour device. \nNow login w

In [ ]:
content='To understand how to answer a user\'s context-based search request for "helpful AI assistant" using Synapse Search Engine, you need to follow the steps below:\n\n1. Read the text provided in the context and identify the keywords or phrases relevant to your query.\n2. Check if the searched term is mentioned anywhere else in the article or within the document. This will help you narrow down your search results.\n3. Check the website\'s URL for any other relevant pages that might be useful.\n4. Consider the date of publication and see if it matches the time frame for your query.\n5. Look for related keywords or phrases in the article to help you understand the context further.\n6. If necessary, use additional keyword research tools like Google Trends or SEMrush to gain more insights into the topic and potential search volume.\n7. Use Synapse Search Engine\'s advanced features, such as synonyms, reverse lookup, or phrase match, to refine your results and find the most relevant answers.\n8. If you\'re still unsure about what to do next, consult with a professional who can help you identify the best approach for your specific context-based search request.' additional_kwargs={} response_metadata={'model': 'tinyllama', 'created_at': '2024-11-28T07:40:44.619601Z', 'done': True, 'done_reason': 'stop', 'total_duration': 850243143700, 'load_duration': 1181269500, 'prompt_eval_count': 1570, 'prompt_eval_duration': 706653000000, 'eval_count': 259, 'eval_duration': 142126000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-2a0df05e-e8af-4455-a8cd-fd6f2dffdd05-0' usage_metadata={'input_tokens': 1570, 'output_tokens': 259, 'total_tokens': 1829}

How to execute scan functionality..?
content='**The query is relevant to the document.\n\nTo execute the scan functionality, follow these steps:\n\n1. Tap your phone to scan a file to yourself (via email), email distribution lists, and popular cloud storage services.\n2. Create contactless copy jobs from your mobile phone and save your favorite copy settings.\n3. After installing the Mobile application, go to Step 6 in the setup process: To get that ID go to the Browser from the same mobile and search for UUID GENERATOR or uuidgenerator.net and copy the latest UUID and Paste it on that Step 5 page or paste it on that Print, Share/Display and Checkin and go to the bottom and save that given id.\n\nNote that the document does not provide a detailed explanation of how to execute the scan functionality. However, it mentions that you can tap your phone to scan a file and create contactless copy jobs from your mobile phone.**' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-12-02T11:37:09.043607Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4387881640100, 'load_duration': 510538600, 'prompt_eval_count': 1517, 'prompt_eval_duration': 4003514000000, 'eval_count': 183, 'eval_duration': 383848000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-c07c60cb-09d8-480a-9282-973b7d20b17d-0' usage_metadata={'input_tokens': 1517, 'output_tokens': 183, 'total_tokens': 1700}

In [ ]:
content='The query is relevant to the document.\n\nTo execute Scan, follow these steps:\n\n1. Tap your phone to scan a file to yourself (via email), email distribution lists, and popular cloud storage services.\n2. The mobile application will automatically detect the scanned document and allow you to save it to your preferred location.\n\nNote that the exact steps may vary depending on the specific device and operating system being used.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-12-02T12:22:32.3179432Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1986017443500, 'load_duration': 12094291700, 'prompt_eval_count': 1516, 'prompt_eval_duration': 1838643000000, 'eval_count': 82, 'eval_duration': 135270000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-ce774771-29fb-4421-be55-aff9e2122983-0' usage_metadata={'input_tokens': 1516, 'output_tokens': 82, 'total_tokens': 1598}